In [1]:
import numpy as np
import pandas as pd

In [ ]:
reg_ratios = pd.read_csv("RegularSeasonDetailedResultsRatios.csv")
tour_ratios = pd.read_csv("NCAATourneyDetailedResultsRatios.csv")

# Get columns
all_cols = reg_ratios.columns.tolist()
cols_to_drop = ["Season", "WTeamID", "WScore", "LTeamID", "LScore", "NumTeamWon", "Loc"]
stats_columns = [item for item in all_cols if item not in cols_to_drop]

# seasons = pd.unique(reg_ratios["Season"])
seasons = [2011, 2012, 2013, 2014, 2015, 2016, 2017]
print(seasons)

In [ ]:
team_ids = pd.DataFrame(index=range(7*68), columns=["Season", "TeamID"])
for i in range(len(seasons)):
    season = seasons[i]
    team_ids_for_season = pd.unique(tour_ratios.loc[tour_ratios["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for j in range(len(team_ids_for_season)):
        team_ids.loc[i*68+j, "Season"] = season
        team_ids.loc[i*68+j, "TeamID"] = team_ids_for_season[j]
        
# Create a data sets to hold regular season averages for every team in the tournament for every specified year
reg_season_avg_cols = ["Season", "TeamID"] + stats_columns
reg_season_avgs = pd.DataFrame(index=range(team_ids.shape[0]), columns=reg_season_avg_cols)
reg_season_avgs["Season"] = team_ids["Season"]
reg_season_avgs["TeamID"] = team_ids["TeamID"]

for season in seasons:
    team_ids_for_season = reg_season_avgs.loc[reg_season_avgs["Season"] == season, "TeamID"].tolist()
        
    for team_id in team_ids_for_season:
        games_all = reg_ratios.loc[(reg_ratios["WTeamID"] == team_id) | (reg_ratios["LTeamID"] == team_id)]
        games = games_all.loc[games_all["Season"] == season]
        games.reset_index(inplace=True, drop=True)
        games_stats = games.loc[:, stats_columns]
        for i, row in games.iterrows():
            if ((team_id == row["WTeamID"]) & (row["NumTeamWon"] == 0)) | ((team_id == row["LTeamID"]) & (row["NumTeamWon"] == 1)):
                games_stats.iloc[i] = games_stats.iloc[i].apply(np.reciprocal)
    
        reg_season_avgs.loc[(reg_season_avgs["TeamID"] == team_id) & (reg_season_avgs["Season"] == season), stats_columns] = games_stats.mean(axis=0).ravel()

reg_season_avgs.to_csv("RegularSeasonAverageRatios.csv")

# Now, create all regular season averages for all teams, even if not in tournament

In [2]:
root = "/home/austin/Github/kaggle-ncaa-2018/"
reg_ratios = pd.read_csv(root + "derived_data/ratios/RegularSeasonDetailedResultsRatios.csv")

# Get columns
all_cols = reg_ratios.columns.tolist()
cols_to_drop = ["Season", "WTeamID", "WScore", "LTeamID", "LScore", "NumTeamWon", "Loc"]
stats_columns = [item for item in all_cols if item not in cols_to_drop]

# seasons = pd.unique(reg_ratios["Season"])
seasons = [2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [3]:
reg_season_avg_cols = ["Season", "TeamID"] + stats_columns
avg_df = pd.DataFrame(columns=reg_season_avg_cols)

for season in seasons:
    team_ids_for_season = pd.unique(reg_ratios.loc[reg_ratios["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for team_id in team_ids_for_season:
        avg_df = avg_df.append({"Season": season, "TeamID": team_id}, ignore_index=True)
        
        games_all = reg_ratios.loc[(reg_ratios["WTeamID"] == team_id) | (reg_ratios["LTeamID"] == team_id)]
        games = games_all.loc[games_all["Season"] == season]
        games.reset_index(inplace=True, drop=True)
        games_stats = games.loc[:, stats_columns]
        for i, row in games.iterrows():
            if ((team_id == row["WTeamID"]) & (row["NumTeamWon"] == 0)) | ((team_id == row["LTeamID"]) & (row["NumTeamWon"] == 1)):
                games_stats.iloc[i] = games_stats.iloc[i].apply(np.reciprocal)
                
        avg_df.loc[(avg_df["TeamID"] == team_id) & (avg_df["Season"] == season), stats_columns] = games_stats.mean(axis=0).ravel()
        
# avg_df.to_csv(root + "derived_data/ratios/RegularSeasonAverageStats.csv")

In [4]:
reg_comp = pd.read_csv(root + "derived_data/composite/RegularSeasonCompositeStats.csv")

comp_columns = [item for item in reg_comp.columns.tolist() if item not in ["Season", "TeamID"]]

avg_df_test = avg_df

In [5]:
# print(avg_df.shape)
# print(reg_comp.loc[reg_comp["Season"] > 2010].shape)
# print(comp_columns)

# Add columns
for comp_column in comp_columns:
    avg_df_test[comp_column] = float('NaN')
    
for i in range(avg_df.shape[0]):
    season = avg_df_test.loc[i, "Season"]
    team_id = avg_df_test.loc[i, "TeamID"]
    reg_comp_row = reg_comp.loc[(reg_comp["Season"] == season) & (reg_comp["TeamID"] == team_id)]
    
    if(reg_comp_row.empty):
        continue
    reg_comp_row.reset_index(inplace=True, drop=True)
    reg_comp_row = reg_comp_row.iloc[0]
    
    if i == 1:
        print(reg_comp_row[comp_columns])
        print(type(reg_comp_row[comp_columns]))
    
    avg_df_test.loc[(avg_df_test["Season"] == season) & (avg_df_test["TeamID"] == team_id), comp_columns] = reg_comp_row[comp_columns].ravel()

PIE              0.620869
FG_PCT           0.509927
TURNOVER_RATE    0.162335
OFF_REB_PCT      0.353229
FT_RATE          0.397974
4FACTOR          0.374897
OFF_EFF          1.125615
DEF_EFF          0.993720
ASSIST_RATIO     0.165896
DEF_REB_PCT      0.685301
FT_PCT           0.674232
WINPCT           0.575758
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>


In [6]:
print(avg_df_test)

      Season  TeamID      FGMR      FGAR     FGMR3     FGAR3      FTMR  \
0     2011.0  1228.0  1.187954  1.005628  1.359628  0.983516  0.968278   
1     2011.0  1268.0  1.228494  1.005001  0.914101  0.722489  1.463031   
2     2011.0  1338.0  1.224000  1.028015  1.351587  0.958004  1.705579   
3     2011.0  1400.0  1.225046  1.015261  1.843969  1.094936  1.565692   
4     2011.0  1104.0  1.257320  1.063398  0.934680  0.900063  1.315700   
5     2011.0  1110.0  1.069533  1.000173  1.192952  1.220678  1.166023   
6     2011.0  1111.0  0.933558  0.958211  1.255630  1.120170  1.701923   
7     2011.0  1114.0  1.021150  1.053878  1.756854  1.007476  1.241077   
8     2011.0  1122.0  1.105515  1.027322  1.173888  0.880473  1.376097   
9     2011.0  1124.0  1.132010  0.984523  0.951032  0.880507  1.650068   
10    2011.0  1130.0  1.000726  0.954461  1.622575  1.286747  1.351647   
11    2011.0  1135.0  0.899676  0.922758  1.379689  1.176589  1.362188   
12    2011.0  1140.0  1.168364  1.0495

In [7]:
avg_df_test.to_csv(root + "derived_data/ratios/RegularSeasonAverageRatiosWithComposites.csv")